In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [1]:
daytime = pd.read_csv("DateOfDay01.csv")

NameError: name 'pd' is not defined

### 抗生素

In [3]:
def split_row(str_un):
    # input type is str.split()
    row_temp = []
    if(str_un[0] != "藥典明細"):
        return row_temp
    # 目前用藥遺囑

    for i in range(len(str_un)):
        try:
            float(str_un[i])
            break
        except:
            if('過敏' in str_un[i]):
                break
            pass

    row_temp.append(" ".join(str_un[:i]))

    # 劑量 單位 途徑 頻次
    i += 4

    # 生效時間
    row_temp.append(" ".join(str_un[i:i+2]))
    i+=2

    # 停用時間
    row_temp.append(" ".join(str_un[i:i+2]))
    i+=2
    
    return row_temp

def read_data(path, file_str):
    data = pd.read_excel(path, sheet_name="4.處方藥物")
    if(data.columns.shape[0] == 1):
        columns = ['目前用藥醫囑', '反序 生效時間', '停用時間']
        dataset = []
        for row_data in data.values:
            str_un = row_data[0].split()
            dataset.append(split_row(str_un))
        data_csv = pd.DataFrame(dataset)
        data_csv.columns = columns
        data_csv = data_csv.dropna(how='all')
        return data_csv
    
    else:
        data.columns = data[0:1].values[0]
        columns = []
        columns = [data.columns[0], data.columns[5], data.columns[6]]
        data_csv = data[1:]
        data_csv = data_csv[columns]
        data_csv = data_csv.dropna(how='all')
        return data_csv

In [11]:
def clean_data_filter(day_count, pattern, file_str, path, k):
    date_format = "%Y-%m-%d"
    date_format1 = "%Y/%m/%d"
    
    data = read_data(path, file_str)
    target = data[data[data.columns[0]].str.lower().str.contains(pattern)]
    if(target.shape[0] != 0):
        day_icu = datetime.strptime(i[1], date_format)
        day_icu_end = day_icu + timedelta(days=27) # 如果更改成4週，則改為27

        for j in target.values:
            day_start = j[1].split()[0]
            day_start = str(int(day_start[:3]) + 1911) + "/" + day_start[-5:] # 轉換到西元
            day_start = datetime.strptime(day_start, date_format1)
            day_end = j[2].split()[0]
            day_end = str(int(day_end[:3]) + 1911) + "/" + day_end[-5:]
            day_end = datetime.strptime(day_end, date_format1)

#                 print('-------------')
#                 print('start:', day_start)
#                 print('end:', day_end)
#                 print('icu_in:', day_icu)
#                 print('-------------')

            temp = []
            name = j[0]
            pid = i[0]
            if((day_start >= day_icu) & (day_icu_end >= day_end)):
                temp = [name, pid, day_icu, day_start, day_end]
                day_count.loc[k] = temp
                k += 1
            elif((day_start < day_icu) & (day_icu <= day_end)):
                # icu前就吃藥，吃到icu期間停止
                if(day_icu_end >= day_end):
                    temp = [name, pid, day_icu, day_icu, day_end]
                else:
                    temp = [name, pid, day_icu, day_icu, day_icu_end]
                day_count.loc[k] = temp
                k += 1
            elif((day_start < day_icu_end) & (day_icu_end <= day_end)):
                # icu期間吃藥，並吃到icu六週後
                if(day_icu <= day_start):
                    temp = [name, pid, day_icu, day_start, day_icu_end]
                else:
                    temp = [name, pid, day_icu, day_icu, day_icu_end]
                day_count.loc[k] = temp
                k += 1
            else:
                pass
    else:
        pass
    return day_count, k

### 各抗生素使用天數

In [68]:
#### clean DF settting ####
Carbapenem = "INVANZ Mepem TIENAM FINIBAX MeROpem".lower().split()
Colistin = "Colimycin Free-COLImycin".lower().split()
Anti_fungal = "ERAXIS CANCIDAS MYCAMINE VFEND FUNGIZONE Ambisome Ampholipad Liposome".lower().split()
Anti_CMV = "ValCYTE Cymevene".lower().split()
Anti_MRSA = "U-Vanco Vancomycin VANCOMYCIN TEICOD CUBICIN TARGOCID Zyvox".lower().split()
anti_set = [Carbapenem, Colistin, Anti_fungal, Anti_CMV, Anti_MRSA]
anti_day_count = pd.DataFrame(columns=['name', 'pid', 'icu_day', 'start_day', 'end_day'])
k = 0
anti_set = [item for sublist in anti_set for item in sublist]
pattern = "|".join(anti_set)
date_format = "%Y-%m-%d"
date_format1 = "%Y/%m/%d"

for i in daytime.values:
    file_str = "t" + str(i[0]).zfill(4)
    path = file_str + ".xlsx"
    try:
        data = read_data(path, file_str)
        target = data[data[data.columns[0]].str.lower().str.contains(pattern)]
        if(target.shape[0] != 0):
            day_icu = datetime.strptime(i[1], date_format)
            day_icu_end = day_icu + timedelta(days=41) # 如果更改成4週，則改為27

            for j in target.values:
                day_start = j[1].split()[0]
                day_start = str(int(day_start[:3]) + 1911) + "/" + day_start[-5:] # 轉換到西元
                day_start = datetime.strptime(day_start, date_format1)

                day_end = j[2].split()[0]
                day_end = str(int(day_end[:3]) + 1911) + "/" + day_end[-5:]
                day_end = datetime.strptime(day_end, date_format1)

#                 print('-------------')
#                 print('start:', day_start)
#                 print('end:', day_end)
#                 print('icu_in:', day_icu)
#                 print('-------------')
                temp = []
                name = j[0]
                pid = i[0]
                if((day_start >= day_icu) & (day_icu_end >= day_end)):
                    temp = [name, pid, day_icu, day_start, day_end]
                    anti_day_count.loc[k] = temp
                    k += 1
                elif((day_start < day_icu) & (day_icu <= day_end)):
                    # icu前就吃藥，吃到icu期間停止
                    if(day_icu_end >= day_end):
                        temp = [name, pid, day_icu, day_icu, day_end]
                    else:
                        temp = [name, pid, day_icu, day_icu, day_icu_end]
                    anti_day_count.loc[k] = temp
                    k += 1
                elif((day_start < day_icu_end) & (day_icu_end <= day_end)):
                    # icu期間吃藥，並吃到icu六週後
                    if(day_icu <= day_start):
                        temp = [name, pid, day_icu, day_start, day_icu_end]
                    else:
                        temp = [name, pid, day_icu, day_icu, day_icu_end]
                    anti_day_count.loc[k] = temp
                    k += 1
                else:
                    pass
        else:
            pass
    except:
        continue
anti_day_count
# anti_day_count.to_csv("./clean_data/anti_day_count.csv", index=None)


,name,pid,icu_day,start_day,end_day
0,藥典明細 COLImycin inj-66.8 [已停用] 抗生素用藥,5,2017-12-17,2018-01-17,2018-01-23
1,藥典明細 COLImycin inj-66.8 [已執行] 抗生素用藥,5,2017-12-17,2018-01-17,2018-01-17
2,藥典明細 COLImycin inj-66.8 [已停用] 抗生素用藥,5,2017-12-17,2018-01-17,2018-01-23
3,藥典明細 500mg Tienam inj [已停用] 抗生素用藥,5,2017-12-17,2018-01-09,2018-01-10
4,藥典明細 500mg Tienam inj [已停用] 抗生素用藥,5,2017-12-17,2018-01-07,2018-01-09
5,藥典明細 COLImycin inj-66.8 [已過期] 抗生素用藥,12,2017-12-28,2018-01-08,2018-01-20
6,藥典明細 COLImycin inj-66.8 [已過期] 抗生素用藥,12,2017-12-28,2018-01-07,2018-01-22
7,藥典明細 COLImycin inj-66.8 [已執行] 抗生素用藥,12,2017-12-28,2018-01-06,2018-01-06
8,藥典明細 500mg Tienam inj [已停用] 抗生素用藥,15,2017-11-26,2017-12-18,2017-12-20
9,藥典明細 Ambisome inj 50mg [已停用] 此為高警訊藥品 抗生素用藥,15,2017-11-26,2017-12-15,2017-12-21


### 升壓降壓藥

In [69]:
#### clean DF settting ####
antihypertensive = "Adrenaline Levophed EPIrenamin Epinephrine Norepinephrine PITRESSIN Dopamin Gipamine".lower().split()
antihypertensive = "|".join(antihypertensive)
hyper_set_all = ["adrenaline", "levophed", "epirenamin", "epinephrine", "norepinephrine", "pitressin", "dopamin" ,"gipamine"]
hyper_day_count = pd.DataFrame(columns=['name', 'pid', 'icu_day', 'start_day', 'end_day'])
k = 0
pattern = "|".join(hyper_set_all)

for i in daytime.values:
    file_str = "t" + str(i[0]).zfill(4)
    path = file_str + ".xlsx"
    try:
        data = read_data(path, file_str)
        target = data[data[data.columns[0]].str.lower().str.contains(pattern)]
        if(target.shape[0] != 0):
            day_icu = datetime.strptime(i[1], date_format)
            day_icu_end = day_icu + timedelta(days=27)

            for j in target.values:
                day_start = j[1].split()[0]
                day_start = str(int(day_start[:3]) + 1911) + "/" + day_start[-5:] # 轉換到西元
                day_start = datetime.strptime(day_start, date_format1)

                day_end = j[2].split()[0]
                day_end = str(int(day_end[:3]) + 1911) + "/" + day_end[-5:]
                day_end = datetime.strptime(day_end, date_format1)

                temp = []
                name = j[0]
                pid = i[0]
                if((day_start >= day_icu) & (day_icu_end >= day_end)):
                    temp = [name, pid, day_icu, day_start, day_end]
                    hyper_day_count.loc[k] = temp
                    k += 1
                elif((day_start < day_icu) & (day_icu <= day_end)):
                    # icu前就吃藥，吃到icu期間停止
                    if(day_icu_end >= day_end):
                        temp = [name, pid, day_icu, day_icu, day_end]
                    else:
                        temp = [name, pid, day_icu, day_icu, day_icu_end]
                    hyper_day_count.loc[k] = temp
                    k += 1
                elif((day_start < day_icu_end) & (day_icu_end <= day_end)):
                    # icu期間吃藥，並吃到icu六週後
                    if(day_icu <= day_start):
                        temp = [name, pid, day_icu, day_start, day_icu_end]
                    else:
                        temp = [name, pid, day_icu, day_icu, day_icu_end]
                    hyper_day_count.loc[k] = temp
                    k += 1
                else:
                    pass
        else:
            pass
    except:
        continue
hyper_day_count

,name,pid,icu_day,start_day,end_day
0,藥典明細 Epinephrine inj 1mg [已執行] 此為高警訊藥品 慎防外滲藥,2,2017-12-20,2018-01-16,2018-01-16
1,藥典明細 Levophed inj 4mg [已停用] 此為高警訊藥品 慎防外滲藥,3,2017-12-20,2018-01-09,2018-01-10
2,藥典明細 Levophed inj 4mg [已過期] 此為高警訊藥品 慎防外滲藥,3,2017-12-20,2018-01-05,2018-01-08
3,藥典明細 Levophed inj 4mg [已停用] 此為高警訊藥品 慎防外滲藥,3,2017-12-20,2018-01-05,2018-01-05
4,藥典明細 Levophed inj 4mg [已停用] 此為高警訊藥品 慎防外滲藥,3,2017-12-20,2018-01-03,2018-01-05
5,藥典明細 Levophed inj 4mg [已停用] 此為高警訊藥品 慎防外滲藥,3,2017-12-20,2018-01-02,2018-01-03
6,藥典明細 Levophed inj 4mg [已停用] 此為高警訊藥品 慎防外滲藥,3,2017-12-20,2017-12-30,2018-01-02
7,藥典明細 Levophed inj 4mg [已停用] 此為高警訊藥品 慎防外滲藥,3,2017-12-20,2017-12-29,2017-12-30
8,藥典明細 Levophed inj 4mg [已停用] 此為高警訊藥品 慎防外滲藥,3,2017-12-20,2017-12-29,2017-12-29
9,藥典明細 Levophed inj 4mg [已停用] 此為高警訊藥品 慎防外滲藥,3,2017-12-20,2017-12-28,2017-12-29


### 計算有抗生素、升壓藥次數

In [70]:
anti_day_count = pd.read_csv("clean_data/anti_day_count.csv")
hyper_day_count = pd.read_csv("clean_data/hyper_day_count.csv")

In [114]:
#### column setting ####
columns_anti = ["Carbapenem", "Colistin", "Anti_CMV", "Anti_fungal", "Anti_MRSA"]
anti_dict = [Carbapenem, Colistin, Anti_fungal, Anti_CMV, Anti_MRSA]

# set anti dict
anti_dict = dict()
anti_listset = [Carbapenem, Colistin, Anti_fungal, Anti_CMV, Anti_MRSA]
for label, items in zip(columns_anti, anti_listset):
    for item in items:
        anti_dict[item] = label
        
columns = []
for j in columns_anti:
    for i in range(1,5):
        columns.append(j + '_W' + str(i))
anti_days = pd.DataFrame(0, index = daytime.values[:,0], columns=columns)


# set hyper dict 
hyper_set_all = ["adrenaline", "levophed", "epirenamin", "epinephrine", "norepinephrine", "pitressin", "dopamin" ,"gipamine"]
columns = []
for j in hyper_set_all:
    for i in range(1,5):
       columns.append(j + '_W' + str(i)) 
hyper_days = pd.DataFrame(0, index=daytime.values[:,0], columns=columns)


In [72]:
anti_day_count.sample(2)

,name,pid,icu_day,start_day,end_day
163,藥典明細 U-Vanco inj 1gm [逾期未審停用] 此為高副作用藥品 抗生素用藥,633,2016-09-06,2016-09-12,2016-09-12
294,藥典明細 COLImycin inj-66.8 [已過期] 抗生素用藥,1156,2013-08-23,2013-09-13,2013-09-16


In [118]:
import numpy as np
date_format = "%Y-%m-%d"
# anti_days.loc[1,'Carbapenem_W1']
for i in anti_day_count.values:
    temp = np.zeros(4, dtype=int)
    day_icu = datetime.strptime(i[2], date_format)
    day_start = datetime.strptime(i[3], date_format)
    day_end = datetime.strptime(i[4], date_format)
    
    dif_start = (day_start - day_icu).days + 1
    dif_end = (day_end - day_icu).days + 1
    dif = (day_end - day_start).days + 1
#     print('start:',dif_start, " dif:",dif, " end:", dif_end)
#     print(i)
    
    week_start = dif_start // 7 
    week_count = dif_start % 7
    if(week_count == 0):
        week_start -= 1
        week_count = 7
    
    # 計算第一週
    if(dif + week_count > 7):
        for dif_count in range(week_count, 8): # 1~7
            temp[week_start] += 1
            dif -= 1
    else:
        for dif_count in range(week_count, dif + week_count): # 1~7
            temp[week_start] += 1
            dif -= 1
    week_start += 1
    for count in range(dif, 0, -1):
        if(temp[week_start] < 7):
            temp[week_start] += 1
        else:
            week_start += 1
            temp[week_start] += 1
    
    
    for k in anti_set:
        if(k in i[0].lower()):
            col = anti_dict[k]
            col_set = []
            for count in range(1, 5):
                col_set.append(col + '_W' + str(count))

            for count in range(4):
                if(temp[count] > anti_days.loc[i[1], col_set][count]):
                    anti_days.loc[i[1], col_set[count]] = temp[count]
anti_days['sum'] = anti_days.sum(axis=1)

In [125]:
anti_days.to_csv("clean_data/anti_days.csv", index=None)

In [126]:
import numpy as np
date_format = "%Y-%m-%d"
# anti_days.loc[1,'Carbapenem_W1']
for i in hyper_day_count.values:
#     print(i[1])
    temp = np.zeros(4, dtype=int)
    day_icu = datetime.strptime(i[2], date_format)
    day_start = datetime.strptime(i[3], date_format)
    day_end = datetime.strptime(i[4], date_format)
    
    dif_start = (day_start - day_icu).days + 1
    dif_end = (day_end - day_icu).days + 1
    dif = (day_end - day_start).days + 1
#     print('start:',dif_start, " dif:",dif, " end:", dif_end)
#     print(i)
    
    week_start = dif_start // 7 
    week_count = dif_start % 7
    if(week_count == 0):
        week_start -= 1
        week_count = 7
    
    # 計算第一週
    if(dif + week_count > 7):
        for dif_count in range(week_count, 8): # 1~7
            temp[week_start] += 1
            dif -= 1
    else:
        for dif_count in range(week_count, dif + week_count): # 1~7
            temp[week_start] += 1
            dif -= 1
    week_start += 1
    for count in range(dif, 0, -1):
        if(temp[week_start] < 7):
            temp[week_start] += 1
        else:
            week_start += 1
            temp[week_start] += 1
#     print(temp)
#     print('--------------------')
    
#     break
    
    for j in range(len(hyper_set_all)):
        # 設置欄位名
        col = hyper_set_all[j]  
        col_set = []
        for count in range(1, 5):
            col_set.append(col + '_W' + str(count))

        for k in hyper_set_all[j]:
            if(k.lower() in i[0].lower()):
                for count in range(4):
                    if(temp[count] > hyper_days.loc[i[1], col_set][count]):
                        hyper_days.loc[i[1], col_set[count]] = temp[count]
hyper_days
# hyper_days['sum'] = hyper_days.sum(axis=1)

,adrenaline_W1,adrenaline_W2,adrenaline_W3,adrenaline_W4,levophed_W1,levophed_W2,levophed_W3,levophed_W4,epirenamin_W1,epirenamin_W2,...,pitressin_W3,pitressin_W4,dopamin_W1,dopamin_W2,dopamin_W3,dopamin_W4,gipamine_W1,gipamine_W2,gipamine_W3,gipamine_W4
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0,1,0,0,...,0,1,0,0,0,1,0,0,0,1
3,1,0,0,0,1,0,0,0,1,0,...,0,0,1,0,0,0,1,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,3,0,0,0,3,0,0,0,3,0,...,0,0,3,0,0,0,3,0,0,0
6,2,0,0,0,2,0,0,0,2,0,...,0,0,2,0,0,0,2,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,3,0,0,0,3,0,0,0,3,0,...,0,0,3,0,0,0,3,0,0,0
9,0,0,3,1,0,0,3,1,0,0,...,3,1,0,0,3,1,0,0,3,1
10,4,0,0,0,4,0,0,0,4,0,...,0,0,4,0,0,0,4,0,0,0


In [140]:
hyper_set = np.array(hyper_days.columns).reshape(8,4)
hyper_ans = hyper_days[hyper_set[0]].values
for i in hyper_set[1:]:
    hyper_ans += hyper_days[i].values
hyper_ans
columns = []
for i in range(1,5):
   columns.append("hyper" + '_W' + str(i)) 
hyper_days_merge = pd.DataFrame(hyper_ans, columns=columns)
hyper_days_merge[hyper_days_merge > 7] = 7

In [160]:
hyper_days_merge.to_csv("clean_data/hyper_days.csv")

### 3.治療

In [161]:
import pandas as pd
from datetime import datetime ,timedelta
import numpy as np
daytime = pd.read_csv("DateOfDay01.csv")

In [162]:
def getTreatValue(daytime, t):
    # t is the str.split()
    temp = []    
    for i in range(len(t)):
        try:
            day = str(1911 + int(t[i][:3])) + t[i][3:]
            date_start = datetime.strptime(day, date_format)
            break
        except:
            pass
    temp.append(" ".join(t[1:i-2]))
    
    # add pid and icu time
    temp.append(daytime[0])
    temp.append(daytime[1])
    
    # 生效時間
    temp.append(day)
    i += 2

    # 停用時間
    day = str(1911 + int(t[i][:3])) + t[i][3:]
    temp.append(day)
    return temp

In [163]:
date_format = "%Y/%m/%d"
date_format1 = "%Y-%m-%d"
data_set = pd.DataFrame(columns=['治療項目', 'pid', 'icu時間', '生效時間', '停用時間'])
k = 0

for i in daytime.values:
    file_str = "t" + str(i[0]).zfill(4)
    path = file_str + ".xlsx"
    try:
        temp = []
        data = pd.read_excel(path, sheet_name='3.治療')
#         print('----------')
#         print(path, 'success')
#         print('----------')
        if(len(data.columns) == 1):
            
            for value in data.values:
                t = value[0].split()
                try:
#                     print(getTreatValue(i, t))
                    
                    data_set.loc[k] = getTreatValue(i, t)
                    k += 1
                except:
#                     print('f')
                    continue
        else:
            data.columns = data[0:1].values[0]
            data_csv = data[1:]
            data_csv = data_csv.dropna(how='all')
            data_csv = data_csv[[data_csv.columns[0], data_csv.columns[2], data_csv.columns[3]]]
            
            for t in data_csv.values:
                try:
                    temp = []
                    # 治療項目
                    temp.append(" ".join(t[0].split()[1:]))

                    # pid
                    temp.append(i[0])

                    # icu time
                    temp.append(i[1])

                    # start time
                    date = t[1].split()[0]
                    date = str(1911 + int(date[:3])) + date[3:]
                    temp.append(date)

                    # end time
                    date = t[2].split()[0]
                    date = str(1911 + int(date[:3])) + date[3:]
                    temp.append(date)
                    data_set.loc[k] = temp
                    k += 1
                except:
                    continue
    except:
        print('----------')
        print('no file', path)
        print('----------')

data_set

----------
no file t0122.xlsx
----------
----------
no file t0130.xlsx
----------
----------
no file t0175.xlsx
----------
----------
no file t0209.xlsx
----------
----------
no file t0228.xlsx
----------
----------
no file t0253.xlsx
----------
----------
no file t0262.xlsx
----------
----------
no file t0297.xlsx
----------
----------
no file t0299.xlsx
----------
----------
no file t0310.xlsx
----------
----------
no file t0328.xlsx
----------
----------
no file t0349.xlsx
----------
----------
no file t0412.xlsx
----------
----------
no file t0687.xlsx
----------
----------
no file t0734.xlsx
----------
----------
no file t0747.xlsx
----------
----------
no file t0764.xlsx
----------
----------
no file t0801.xlsx
----------
----------
no file t0853.xlsx
----------
----------
no file t0876.xlsx
----------
----------
no file t0882.xlsx
----------
----------
no file t0894.xlsx
----------
----------
no file t0903.xlsx
----------
----------
no file t0967.xlsx
----------
----------
no fi

,治療項目,pid,icu時間,生效時間,停用時間
0,VITAL SIGNS的計價明細 (9901) @CHECK VITAL SIGNS [已停用],1,2017-11-26,2018/01/11,2018/02/02
1,SBP 100 ~ 140 mmHg.的計價明細 (9992) @Keep SBP 100 ...,1,2017-11-26,2017/11/26,2018/02/02
2,(DAYTIME: PS) keep SPO2 > 95%的計價明細 (9886) @WEA...,1,2017-11-26,2018/01/19,2018/02/02
3,Check Blood Sugar的計價明細 (9928) Quick Check Bloo...,1,2017-11-26,2017/12/28,2018/02/02
4,PUPILS SIZE LIGHT FLEX的計價明細 (9918) @GCS PUPILS...,1,2017-11-26,2018/01/02,2018/02/02
5,NS ROUTINE的計價明細 (9902) @ON NS ROUTINE [已停用],1,2017-11-26,2018/01/11,2018/02/02
6,D <10cm(S) (Body)的計價明細 (9879) C D <10cm(S) (Bo...,1,2017-11-26,2018/01/25,2018/02/02
7,D <10cm(S) (Body)的計價明細 (9915) C D <10cm(S) (Bo...,1,2017-11-26,2018/01/03,2018/02/02
8,Foley Cath.& Care的計價明細 (9908) @Keeping Foley C...,1,2017-11-26,2018/01/08,2018/02/02
9,Check Blood Sugar的計價明細 (9966) Quick Check Bloo...,1,2017-11-26,2017/12/01,2018/02/02


In [164]:
# # read
data_set = pd.read_csv('Treat_all.csv')
data_set

# # write
# data_set.to_csv('Treat_all.csv', index=None)

,治療項目,pid,icu時間,生效時間,停用時間
0,VITAL SIGNS的計價明細 (9901) @CHECK VITAL SIGNS [已停用],1,2017-11-26,2018/01/11,2018/02/02
1,SBP 100 ~ 140 mmHg.的計價明細 (9992) @Keep SBP 100 ...,1,2017-11-26,2017/11/26,2018/02/02
2,(DAYTIME: PS) keep SPO2 > 95%的計價明細 (9886) @WEA...,1,2017-11-26,2018/01/19,2018/02/02
3,Check Blood Sugar的計價明細 (9928) Quick Check Bloo...,1,2017-11-26,2017/12/28,2018/02/02
4,PUPILS SIZE LIGHT FLEX的計價明細 (9918) @GCS PUPILS...,1,2017-11-26,2018/01/02,2018/02/02
5,NS ROUTINE的計價明細 (9902) @ON NS ROUTINE [已停用],1,2017-11-26,2018/01/11,2018/02/02
6,D <10cm(S) (Body)的計價明細 (9879) C D <10cm(S) (Bo...,1,2017-11-26,2018/01/25,2018/02/02
7,D <10cm(S) (Body)的計價明細 (9915) C D <10cm(S) (Bo...,1,2017-11-26,2018/01/03,2018/02/02
8,Foley Cath.& Care的計價明細 (9908) @Keeping Foley C...,1,2017-11-26,2018/01/08,2018/02/02
9,Check Blood Sugar的計價明細 (9966) Quick Check Bloo...,1,2017-11-26,2017/12/01,2018/02/02


In [165]:
hemodialysis = ['HD', 'Dialysis', 'hemodialysis', 'Hemodialysis'] 
DNR = ['DNR', 'DO-NOT-RESUSCITATE']
# 註：抓keyword，有出現hemodialysis的就算一次
# Critical dialysis [次數 (week-1, 2, 3, 4, 5, 6)]
# blood transfusion [次數 (week-1, 2, 3, 4, 5, 6)]
# DO-NOT-RESUSCITATE/DNR[次數 (week-1, 2, 3, 4, 5, 6)，第幾天] 
# Palliative care Family conference Fee[次數 (week-1, 2, 3, 4, 5, 6)，第幾天]
# RCC routine [總天數]
# High frequency chest wall compression [第幾天]
# CPR [次數 (week-1, 2, 3, 4, 5, 6)，第幾天]
# On Ventilator [總天數]
# Tracheostomy [有無]
# Tapping [有無]
treat_col = ['Critical dialysis', 'blood transfusion', 'Palliative care Family conference Fee', 'CPR']
dayWhen_col = ['Palliative care Family conference Fee', 'High frequency chest wall compression', 'CPR']
dayAll_col = ['RCC routine', 'On Ventilator']
treatWhether_col = ['Tracheostomy', 'Tapping']
foley = 'Keeping Foley'

In [166]:
foley_set = data_set[data_set[data_set.columns[0]].str.contains(foley)]

In [167]:
pattern = "|".join(hemodialysis)
hemodialysis_set = data_set[data_set[data_set.columns[0]].str.contains(pattern)]

pattern = "|".join(DNR).lower()
DNR_set = data_set[data_set[data_set.columns[0]].str.lower().str.contains(pattern)]

In [168]:
# count week
pattern = "|".join(treat_col).lower()
treat_col_set = data_set[data_set[data_set.columns[0]].str.lower().str.contains(pattern)]

# return day sum
pattern = "|".join(dayAll_col).lower()
dayAll_col_set = data_set[data_set[data_set.columns[0]].str.lower().str.contains(pattern)]

# find when first
pattern = "|".join(dayWhen_col).lower()
dayWhen_col_set = data_set[data_set[data_set.columns[0]].str.lower().str.contains(pattern)]

# whether have
pattern = "|".join(treatWhether_col).lower()
treatWhether_col_set = data_set[data_set[data_set.columns[0]].str.lower().str.contains(pattern)]

In [169]:
date_format = "%Y/%m/%d"
date_format1 = "%Y-%m-%d"

foley_set['icu時間'] = pd.to_datetime(foley_set['icu時間'].values, format=date_format1)
foley_set['生效時間'] = pd.to_datetime(foley_set['生效時間'].values, format=date_format)
foley_set['停用時間'] = pd.to_datetime(foley_set['停用時間'].values, format=date_format)

foley_set['icu時間_out'] = foley_set['icu時間'] + timedelta(days=27)
foley_set = foley_set[((foley_set['生效時間'] >= foley_set['icu時間']) & (foley_set['生效時間'] <= foley_set['icu時間_out'])) | ((foley_set['停用時間'] >= foley_set['icu時間']) & (foley_set['停用時間'] <= foley_set['icu時間_out']))]

foley_set['days'] = foley_set['停用時間'] - foley_set['生效時間'] + timedelta(days=1)

col_set = []
for count in range(1, 5):
    col_set.append('foley' + '_W' + str(count))
foley_days = pd.DataFrame(0, index=daytime.No, columns = col_set)

for i in foley_set.values:
    temp = np.zeros(4, dtype=int)
    day_icu, day_start, day_end, day_icu_out = i[2], i[3], i[4], i[5]
    if(day_start < day_icu):
        day_start = day_icu
    if(day_end > day_icu_out):
        day_end = day_icu_out
    
    dif_start = (day_start - day_icu).days + 1
    dif_end = (day_end - day_icu).days + 1
    dif = (day_end - day_start).days + 1
    
    week_start = dif_start // 7 
    week_count = dif_start % 7
    if(week_count == 0):
        week_start -= 1
        week_count = 7
    
    # 計算第一週
    if(dif + week_count > 7):
        for dif_count in range(week_count, 8): # 1~7
            temp[week_start] += 1
            dif -= 1
    else:
        for dif_count in range(week_count, dif + week_count): # 1~7
            temp[week_start] += 1
            dif -= 1
    week_start += 1
    for count in range(dif, 0, -1):
        if(temp[week_start] < 7):
            temp[week_start] += 1
        else:
            week_start += 1
            temp[week_start] += 1

    foley_days.loc[i[1]] = temp
foley_days.to_csv('./clean_data/foley_days.csv', index=None)

/Users/ys/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/ys/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/Users/ys/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: htt

In [170]:
date_format = "%Y/%m/%d"
date_format1 = "%Y-%m-%d"

DNR_set['icu時間'] = pd.to_datetime(DNR_set['icu時間'].values, format=date_format1)
DNR_set['生效時間'] = pd.to_datetime(DNR_set['生效時間'].values, format=date_format)
DNR_set['停用時間'] = pd.to_datetime(DNR_set['停用時間'].values, format=date_format)

# 取42天內的資料
DNR_set['icu時間_out'] = DNR_set['icu時間'] + timedelta(days=27)
DNR_set = DNR_set[((DNR_set['生效時間'] >= DNR_set['icu時間']) & (DNR_set['生效時間'] <= DNR_set['icu時間_out'])) | ((DNR_set['停用時間'] >= DNR_set['icu時間']) & (DNR_set['停用時間'] <= DNR_set['icu時間_out']))]

# 設置資料欄位
col_set = []
for count in range(1, 5):
    col_set.append('DO-NOT-RESUSCITATE' + '_W' + str(count))
DNR_days = pd.DataFrame(0, index=daytime.No, columns = col_set)

for i in DNR_set.values:
    temp = np.zeros(4, dtype=int)
    day_icu, day_start, day_end, day_icu_out = i[2], i[3], i[4], i[5]
    if(day_start < day_icu):
        day_start = day_icu
    if(day_end > day_icu_out):
        day_end = day_icu_out
    
    dif_start = (day_start - day_icu).days + 1
    dif_end = (day_end - day_icu).days + 1
    dif = (day_end - day_start).days + 1
    
    week_start = dif_start // 7 
    week_count = dif_start % 7
    if(week_count == 0):
        week_start -= 1
        week_count = 7
    
    # 計算第一週
    if(dif + week_count > 7):
        for dif_count in range(week_count, 8): # 1~7
            temp[week_start] += 1
            dif -= 1
    else:
        for dif_count in range(week_count, dif + week_count): # 1~7
            temp[week_start] += 1
            dif -= 1
    week_start += 1
    for count in range(dif, 0, -1):
        if(temp[week_start] < 7):
            temp[week_start] += 1
        else:
            week_start += 1
            temp[week_start] += 1

    DNR_days.loc[i[1]] = temp
DNR_days

/Users/ys/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/ys/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/Users/ys/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: htt

,DO-NOT-RESUSCITATE_W1,DO-NOT-RESUSCITATE_W2,DO-NOT-RESUSCITATE_W3,DO-NOT-RESUSCITATE_W4
No,,,,
1,0,0,0,0
2,0,0,1,7
3,0,0,0,0
4,0,0,0,0
5,0,0,0,0
6,0,0,0,0
7,0,0,0,0
8,7,7,7,7
9,0,0,0,0


In [171]:
date_format = "%Y/%m/%d"
date_format1 = "%Y-%m-%d"

hemodialysis_set['icu時間'] = pd.to_datetime(hemodialysis_set['icu時間'].values, format=date_format1)
hemodialysis_set['生效時間'] = pd.to_datetime(hemodialysis_set['生效時間'].values, format=date_format)
hemodialysis_set['停用時間'] = pd.to_datetime(hemodialysis_set['停用時間'].values, format=date_format)

# 取42天內的資料
hemodialysis_set['icu時間_out'] = hemodialysis_set['icu時間'] + timedelta(days=27)
hemodialysis_set = hemodialysis_set[((hemodialysis_set['生效時間'] >= hemodialysis_set['icu時間']) & (hemodialysis_set['生效時間'] <= hemodialysis_set['icu時間_out'])) | ((hemodialysis_set['停用時間'] >= hemodialysis_set['icu時間']) & (hemodialysis_set['停用時間'] <= hemodialysis_set['icu時間_out']))]

# 設置資料欄位
col_set = []
for count in range(1, 5):
    col_set.append('hemodialysis' + '_W' + str(count))
hemodialysis_days = pd.DataFrame(0, index=daytime.No, columns = col_set)

for i in hemodialysis_set.values:
    temp = np.zeros(4, dtype=int)
    day_icu, day_start, day_end, day_icu_out = i[2], i[3], i[4], i[5]
    if(day_start < day_icu):
        day_start = day_icu
    if(day_end > day_icu_out):
        day_end = day_icu_out
    
    dif_start = (day_start - day_icu).days + 1
    dif_end = (day_end - day_icu).days + 1
    dif = (day_end - day_start).days + 1
#     print('start:',dif_start, " dif:",dif, " end:", dif_end)
#     print(i)
#     break
    
    week_start = dif_start // 7 
    week_count = dif_start % 7
    if(week_count == 0):
        week_start -= 1
        week_count = 7
    
    # 計算第一週
    if(dif + week_count > 7):
        for dif_count in range(week_count, 8): # 1~7
            temp[week_start] += 1
            dif -= 1
    else:
        for dif_count in range(week_count, dif + week_count): # 1~7
            temp[week_start] += 1
            dif -= 1
    week_start += 1
    for count in range(dif, 0, -1):
        if(temp[week_start] < 7):
            temp[week_start] += 1
        else:
            week_start += 1
            temp[week_start] += 1

    hemodialysis_days.loc[i[1]] = temp
hemodialysis_days

/Users/ys/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/ys/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/Users/ys/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: htt

,hemodialysis_W1,hemodialysis_W2,hemodialysis_W3,hemodialysis_W4
No,,,,
1,0,0,0,0
2,0,0,0,0
3,0,1,0,0
4,0,0,0,0
5,1,0,0,0
6,0,0,0,0
7,1,0,0,0
8,0,0,0,0
9,0,0,0,0


In [172]:
date_format = "%Y/%m/%d"
date_format1 = "%Y-%m-%d"

pattern = "|".join(treat_col).lower()
treat_col_set = data_set[data_set[data_set.columns[0]].str.lower().str.contains(pattern)]

treat_col_set['icu時間'] = pd.to_datetime(treat_col_set['icu時間'].values, format=date_format1)
treat_col_set['生效時間'] = pd.to_datetime(treat_col_set['生效時間'].values, format=date_format)
treat_col_set['停用時間'] = pd.to_datetime(treat_col_set['停用時間'].values, format=date_format)

# 取42天內的資料
treat_col_set['icu時間_out'] = treat_col_set['icu時間'] + timedelta(days=27)
treat_col_set = treat_col_set[((treat_col_set['生效時間'] >= treat_col_set['icu時間']) & (treat_col_set['生效時間'] <= treat_col_set['icu時間_out'])) | ((treat_col_set['停用時間'] >= treat_col_set['icu時間']) & (treat_col_set['停用時間'] <= treat_col_set['icu時間_out']))]

columns = []
for j in treat_col:
    for i in range(1,5):
       columns.append(j + '_W' + str(i)) 
treat_col_days = pd.DataFrame(0, index=daytime.values[:,0], columns=columns)

for i in treat_col_set.values:
    temp = np.zeros(4, dtype=int)
    day_icu, day_start, day_end, day_icu_out = i[2], i[3], i[4], i[5]
    if(day_start < day_icu):
        day_start = day_icu
    if(day_end > day_icu_out):
        day_end = day_icu_out
    
    dif_start = (day_start - day_icu).days + 1
    dif_end = (day_end - day_icu).days + 1
    dif = (day_end - day_start).days + 1

    
    week_start = dif_start // 7 
    week_count = dif_start % 7
    if(week_count == 0):
        week_start -= 1
        week_count = 7
    
    # 計算第一週
    if(dif + week_count > 7):
        for dif_count in range(week_count, 8): # 1~7
            temp[week_start] += 1
            dif -= 1
    else:
        for dif_count in range(week_count, dif + week_count): # 1~7
            temp[week_start] += 1
            dif -= 1
    week_start += 1
    for count in range(dif, 0, -1):
        if(temp[week_start] < 7):
            temp[week_start] += 1
        else:
            week_start += 1
            temp[week_start] += 1


    for k in treat_col:
        if(k.lower() in i[0].lower()):
            break
            
    col_set = []
    for count in range(1, 5):
        col_set.append(k + '_W' + str(count))

    
    for count in range(4):
        if(temp[count] > treat_col_days.loc[i[1], col_set][count]):
            treat_col_days.loc[i[1], col_set[count]] = temp[count]
    
treat_col_days

/Users/ys/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/Users/ys/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/ys/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-d

,Critical dialysis_W1,Critical dialysis_W2,Critical dialysis_W3,Critical dialysis_W4,blood transfusion_W1,blood transfusion_W2,blood transfusion_W3,blood transfusion_W4,Palliative care Family conference Fee_W1,Palliative care Family conference Fee_W2,Palliative care Family conference Fee_W3,Palliative care Family conference Fee_W4,CPR_W1,CPR_W2,CPR_W3,CPR_W4
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
5,5,7,7,6,1,1,1,1,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
7,7,7,7,7,1,1,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
9,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0
10,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0


In [173]:
treat_days = hemodialysis_days.join(treat_col_days).join(DNR_days)
treat_days.to_csv('./clean_data/treat_days.csv')

In [174]:
dayAll_col_set['生效時間'] = pd.to_datetime(dayAll_col_set['生效時間'].values, format=date_format)
dayAll_col_set['停用時間'] = pd.to_datetime(dayAll_col_set['停用時間'].values, format=date_format)
dayAll_col_set['sum_day'] = (dayAll_col_set['停用時間'] - dayAll_col_set['生效時間']).dt.days + 1

/Users/ys/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/ys/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/ys/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

In [175]:
dayAll_col_set['icu時間'] = pd.to_datetime(dayAll_col_set['icu時間'].values, format=date_format1)
dayAll_col_set['生效時間'] = pd.to_datetime(dayAll_col_set['生效時間'].values, format=date_format)
dayAll_col_set['停用時間'] = pd.to_datetime(dayAll_col_set['停用時間'].values, format=date_format)
dayAll_col_set['icu時間_out'] = dayAll_col_set['icu時間'] + timedelta(days=27)
dayAll_col_set = dayAll_col_set[((dayAll_col_set['生效時間'] >= dayAll_col_set['icu時間']) & (dayAll_col_set['生效時間'] <= dayAll_col_set['icu時間_out'])) | ((dayAll_col_set['停用時間'] >= dayAll_col_set['icu時間']) & (dayAll_col_set['停用時間'] <= dayAll_col_set['icu時間_out']))]

/Users/ys/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/ys/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/ys/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

In [176]:
RCC_days = dayAll_col_set[dayAll_col_set[dayAll_col_set.columns[0]].str.lower().str.contains('RCC routine'.lower())][['pid', 'sum_day']]
On_Ventilator_days = dayAll_col_set[dayAll_col_set[dayAll_col_set.columns[0]].str.lower().str.contains('On Ventilator'.lower())][['pid', 'sum_day']]

In [177]:
On_Ventilator_days.groupby('pid').sum().astype('int').to_csv("./clean_data/On_Ventilator_sum_days.csv")

In [178]:
RCC_days.groupby('pid').sum().astype('int').to_csv('./clean_data/RCC_sum_days.csv')

In [179]:
RCC_days = pd.read_csv('./clean_data/RCC_sum_days.csv') 
On_Ventilator_days = pd.read_csv("./clean_data/On_Ventilator_sum_days.csv")

In [180]:
treatWhether_col_set['icu時間'] = pd.to_datetime(treatWhether_col_set['icu時間'].values, format=date_format1)
treatWhether_col_set['生效時間'] = pd.to_datetime(treatWhether_col_set['生效時間'].values, format=date_format)
treatWhether_col_set['停用時間'] = pd.to_datetime(treatWhether_col_set['停用時間'].values, format=date_format)
treatWhether_col_set['icu時間_out'] = treatWhether_col_set['icu時間'] + timedelta(days=27)
treatWhether_col_set = treatWhether_col_set[((treatWhether_col_set['生效時間'] >= treatWhether_col_set['icu時間']) & (treatWhether_col_set['生效時間'] <= treatWhether_col_set['icu時間_out'])) | ((treatWhether_col_set['停用時間'] >= treatWhether_col_set['icu時間']) & (treatWhether_col_set['停用時間'] <= treatWhether_col_set['icu時間_out']))]
treatWhether_col_set

/Users/ys/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/ys/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/ys/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

,治療項目,pid,icu時間,生效時間,停用時間,icu時間_out
1118,Tracheostomy Tube & Care的計價明細 (9914) @Keeping ...,11,2018-01-02,2018-01-24,2018-02-13,2018-01-29
1216,Tracheostomy Tube & Care的計價明細 (9942) @Keeping ...,12,2017-12-28,2018-01-15,2018-01-27,2018-01-24
1233,Tapping的計價明細 (9927) Chest Tapping,12,2017-12-28,2018-01-22,2018-01-22,2018-01-24
1265,Tapping的計價明細 (9960) Chest Tapping,12,2017-12-28,2018-01-05,2018-01-05,2018-01-24
1391,Tracheostomy Tube & Care的計價明細 (9939) @Keeping ...,14,2018-01-09,2018-01-22,2018-01-27,2018-02-05
1674,Tapping的計價明細 (9934) Chest Tapping,15,2017-12-15,2017-12-15,2017-12-15,2018-01-11
1857,Tracheostomy Tube & Care的計價明細 (9953) @Keeping ...,17,2018-01-22,2018-01-30,2018-02-26,2018-02-18
1903,CXR two hours after tracheostomy的計價明細 (9954) @...,17,2018-01-22,2018-01-30,2018-01-30,2018-02-18
2378,Tracheostomy Tube & Care的計價明細 (9956) @Keeping ...,22,2018-01-21,2018-01-31,2018-03-02,2018-02-17
3011,Tapping的計價明細 (9933) Chest Tapping,26,2018-01-27,2018-02-21,2018-02-21,2018-02-23


In [181]:
treatWhether_col

['Tracheostomy', 'Tapping']

In [182]:
treatWhether = pd.DataFrame(0, index=daytime.No, columns=treatWhether_col)
for i in treatWhether_col_set.values:
    if(treatWhether_col[0].lower() in i[0].lower()):
        treatWhether.loc[i[1], treatWhether_col[0]] = 1
    elif(treatWhether_col[1].lower() in i[0].lower()):
        treatWhether.loc[i[1], treatWhether_col[1]] = 1

treatWhether.to_csv('./clean_data/treatWhether_days.csv')

In [183]:
date_format = "%Y/%m/%d"
date_format1 = "%Y-%m-%d"

pattern = "|".join(dayWhen_col).lower()
dayWhen_col_set = data_set[data_set[data_set.columns[0]].str.lower().str.contains(pattern)]

dayWhen_col_set['icu時間'] = pd.to_datetime(dayWhen_col_set['icu時間'].values, format=date_format1)
dayWhen_col_set['生效時間'] = pd.to_datetime(dayWhen_col_set['生效時間'].values, format=date_format)
dayWhen_col_set['停用時間'] = pd.to_datetime(dayWhen_col_set['停用時間'].values, format=date_format)

# 取28天內的資料
dayWhen_col_set['icu時間_out'] = dayWhen_col_set['icu時間'] + timedelta(days=27)
dayWhen_col_set = dayWhen_col_set[((dayWhen_col_set['生效時間'] >= dayWhen_col_set['icu時間']) & (dayWhen_col_set['生效時間'] <= dayWhen_col_set['icu時間_out'])) | ((dayWhen_col_set['停用時間'] >= dayWhen_col_set['icu時間']) & (dayWhen_col_set['停用時間'] <= dayWhen_col_set['icu時間_out']))]

columns = dayWhen_col
dayWhen_col_days = pd.DataFrame(29, index=daytime.values[:,0], columns=columns)


for i in dayWhen_col_set.values:
    temp = np.zeros(4, dtype=int)
    day_icu, day_start, day_end, day_icu_out = i[2], i[3], i[4], i[5]
    if(day_start < day_icu):
        day_start = day_icu
    if(day_end > day_icu_out):
        day_end = day_icu_out
    
    dif_start = (day_start - day_icu).days + 1
    
    for k in dayWhen_col:
        if(k.lower() in i[0].lower()):
            if(dif_start < dayWhen_col_days.loc[i[1], k]):
                dayWhen_col_days.loc[i[1], k] = dif_start
dayWhen_col_days = dayWhen_col_days.replace(29,0)
dayWhen_col_days

/Users/ys/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/Users/ys/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/ys/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-d

,Palliative care Family conference Fee,High frequency chest wall compression,CPR
1,0,0,0
2,21,0,0
3,0,0,0
4,18,0,0
5,0,27,0
6,28,12,0
7,0,0,0
8,22,0,0
9,0,6,0
10,19,0,0


In [184]:
dayWhen_col_days.to_csv('./clean_data/dayWhen_days.csv')

In [185]:
date_format = "%Y/%m/%d"
date_format1 = "%Y-%m-%d"

pattern = "|".join(DNR).lower()
DNR_set = data_set[data_set[data_set.columns[0]].str.lower().str.contains(pattern)]

DNR_set['icu時間'] = pd.to_datetime(DNR_set['icu時間'].values, format=date_format1)
DNR_set['生效時間'] = pd.to_datetime(DNR_set['生效時間'].values, format=date_format)
DNR_set['停用時間'] = pd.to_datetime(DNR_set['停用時間'].values, format=date_format)

# 取28天內的資料
DNR_set['icu時間_out'] = DNR_set['icu時間'] + timedelta(days=27)
DNR_set = DNR_set[((DNR_set['生效時間'] >= DNR_set['icu時間']) & (DNR_set['生效時間'] <= DNR_set['icu時間_out'])) | ((DNR_set['停用時間'] >= DNR_set['icu時間']) & (DNR_set['停用時間'] <= DNR_set['icu時間_out']))]

columns = DNR
DNR_days = pd.DataFrame(29, index=daytime.values[:,0], columns=columns)

for i in DNR_set.values:
    temp = np.zeros(4, dtype=int)
    day_icu, day_start, day_end, day_icu_out = i[2], i[3], i[4], i[5]
    if(day_start < day_icu):
        day_start = day_icu
    if(day_end > day_icu_out):
        day_end = day_icu_out
    
    dif_start = (day_start - day_icu).days + 1
    
    for k in DNR:
        if(k.lower() in i[0].lower()):
            if(dif_start < DNR_days.loc[i[1], k]):
                DNR_days.loc[i[1], k] = dif_start
DNR_days

DNR_days = DNR_days.replace(29,0)
DNR_days['DNR1'] = DNR_days['DNR'] + DNR_days['DO-NOT-RESUSCITATE']
DNR_days = DNR_days[['DNR1']]
DNR_days.columns = ['DNR_days']
DNR_days.join(dayWhen_col_days).to_csv('./clean_data/dayWhen_days.csv')
dayWhen_col_days = DNR_days.join(dayWhen_col_days)

/Users/ys/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/Users/ys/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/ys/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-d

In [186]:
On_Ventilator_days.columns = ['No', 'Ventilator_days']
On_Ventilator_days.index = On_Ventilator_days.No

In [187]:
RCC_days.columns = ['No', 'RCC_days']
RCC_days.index = RCC_days.No

In [188]:
Medi_days_sum = RCC_days[['RCC_days']].join(On_Ventilator_days[['Ventilator_days']])

In [189]:
# dayWhen_col_days
test = treat_days.join(Medi_days_sum)

In [190]:
dayWhen_col_days.columns= ['DNR_startDay','Palliative care Family conference Fee_startDay','High frequency chest wall compression_startDay', 'CPR_startDay']

In [191]:
test = test.join(dayWhen_col_days)

In [192]:
test = test.join(treatWhether)
test = test.join(foley_days)

In [193]:
test = test.fillna(0)

In [194]:
test.to_csv('./clean_data/3_treat.csv')
df_34 = test

In [195]:
# anti_days = pd.read_csv("./clean_data/anti_days.csv")
# anti_days = anti_days[anti_days.columns[1:]]
# t1 = pd.read_csv('./clean_data/hyper_days.csv')
# t1 = t1[t1.columns[1:]]
# df_34 = treat3
df_34 = df_34.join(anti_days)
df_34 = df_34.join(hyper_days_merge)
df_34.to_csv("./clean_data/sheet_merge34.csv")

In [198]:
df_34.columns\

Index(['hemodialysis_W1', 'hemodialysis_W2', 'hemodialysis_W3',
       'hemodialysis_W4', 'Critical dialysis_W1', 'Critical dialysis_W2',
       'Critical dialysis_W3', 'Critical dialysis_W4', 'blood transfusion_W1',
       'blood transfusion_W2', 'blood transfusion_W3', 'blood transfusion_W4',
       'Palliative care Family conference Fee_W1',
       'Palliative care Family conference Fee_W2',
       'Palliative care Family conference Fee_W3',
       'Palliative care Family conference Fee_W4', 'CPR_W1', 'CPR_W2',
       'CPR_W3', 'CPR_W4', 'DO-NOT-RESUSCITATE_W1', 'DO-NOT-RESUSCITATE_W2',
       'DO-NOT-RESUSCITATE_W3', 'DO-NOT-RESUSCITATE_W4', 'RCC_days',
       'Ventilator_days', 'DNR_startDay',
       'Palliative care Family conference Fee_startDay',
       'High frequency chest wall compression_startDay', 'CPR_startDay',
       'Tracheostomy', 'Tapping', 'foley_W1', 'foley_W2', 'foley_W3',
       'foley_W4', 'Carbapenem_W1', 'Carbapenem_W2', 'Carbapenem_W3',
       'Carbapenem_W4'

## 5.累計報告

import pandas as pd
import numpy as np
from datetime import datetime, timedelta

daytime = pd.read_csv("DateOfDay01.csv")
date_format = "%Y/%m/%d"
date_format1 = "%Y-%m-%d"

reportColumn = ['ALB', 'BIL.T', 'ALT', 'WBC', 'HGB', 'PLT']
reportColumn_set = set(reportColumn)
columns = []
for i in reportColumn:
    for j in range(1,5):
        columns.append(i+"_mean W"+str(j).zfill(2))
Sheet5_DF = pd.DataFrame(0, index=daytime.No, columns=columns)

for num in daytime.values:
    print(num[0])
    path = "t" + str(num[0]).zfill(4) + ".xlsx"
    data = pd.read_excel(path, sheet_name='5.累積報告')
    data = data.replace('\xa0\xa0', '0')
    #### standard data ####

    data.columns = np.arange(len(data.columns))
    flag = 0
    sheet5 = []
    for i in range(1, len(data)):
        if(data.loc[i,0] == 'DATE'):
            z = data.loc[flag:i-1, :].dropna(how='all', axis=1)
            z.columns = z[0:1].values[0]
            z = z[1:]
            sheet5.append(z)
            flag = i
            
    reportDF = pd.DataFrame(columns=reportColumn)
    for i in sheet5:
        col = set(i.columns) & reportColumn_set
        if(col != set()):
            col = list(col)
            col.append('DATE')
            reportDF = pd.merge(reportDF, i[col], how='outer')
            
    #### transform time to datetime ####
    data_dirty = reportDF
    date_list = []
    for i in data_dirty['DATE']:
        day_start = i.split()[0]
        day_start = str(int(day_start[:3]) + 1911) + "/" + day_start[-4:-2] + "/" + day_start[-2:] # 轉換到西元
        day_start = datetime.strptime(day_start, date_format)
        date_list.append(day_start)
        
    data_dirty['DATE'] = date_list
    data_dirty['icu_time'] = datetime.strptime(num[1], date_format1)
    data_dirty = data_dirty.sort_values(by='DATE')
    data_dirty['icu_time_out'] = data_dirty['icu_time'] + timedelta(days=27)
    data_dirty = data_dirty[((data_dirty['DATE'] >= data_dirty['icu_time']) & (data_dirty['DATE'] <= data_dirty['icu_time_out']))]
    data_dirty['days'] = data_dirty['DATE'] - data_dirty['icu_time']
    
    
    w1 = data_dirty[(data_dirty.days >= timedelta(days=0)) & (data_dirty.days < timedelta(days=7))][reportColumn]
    w2 = data_dirty[(data_dirty.days >= timedelta(days=7)) & (data_dirty.days < timedelta(days=14))][reportColumn]
    w3 = data_dirty[(data_dirty.days >= timedelta(days=14)) & (data_dirty.days < timedelta(days=21))][reportColumn]
    w4 = data_dirty[(data_dirty.days >= timedelta(days=21)) & (data_dirty.days < timedelta(days=28))][reportColumn]
       
    for i, week in zip([w1,w2,w3,w4], np.arange(1,5)):

        i = i.fillna('0')
        for j in reportColumn:
            z = []
            for k in i[j].str.split():
                try:
                    value = float(k[0])
                except:
                    value = 0
                z.append(value)
            i[j] = z
        for j in reportColumn:
            i[j].loc[i[j] == 0] = i[j].mean()    
        std = dict(i.std())

        for temp in std:
            temp_col = temp + "_SD W" + str(week).zfill(2)
            if(np.isnan(std[temp])):
                Sheet5_DF.loc[num[0], temp_col] = 0
            else:
                Sheet5_DF.loc[num[0], temp_col] = std[temp]


# Sheet5_DF = Sheet5_DF.drop('No', axis=1)
reportDF

# Sheet5_DF = Sheet5_DF.drop('No.1', axis=1)
# Sheet5_DF.to_csv("./clean_data/Sheet5.csv")